In [15]:
import pandas as pd
import urllib

In [16]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f6f27e7d4c0>)

In [17]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [18]:
#null 값 삭제

In [19]:
train_data = train_data[~train_data['document'].isnull()]

In [20]:
train_data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


# 형태소 분석
## 빠른 속도 때문에 mecab 활용

In [21]:
from tqdm import tqdm, tqdm_pandas

In [22]:
tqdm_pandas(tqdm())

0it [00:00, ?it/s]


In [23]:
from konlpy.tag import Mecab

In [24]:
text = train_data['document']

In [25]:
mecab = Mecab()

In [26]:
morphs = train_data['document'].progress_apply(lambda x : mecab.morphs(x))

149995it [00:09, 16163.57it/s]


In [27]:
morphs

0                       [아, 더, 빙, ., ., 진짜, 짜증, 나, 네요, 목소리]
1         [흠, ., .., 포스터, 보고, 초딩, 영화, 줄, ., ..., 오버, 연기,...
2                                   [너무, 재, 밓었다그래서보는것을추천한다]
3         [교도소, 이야기, 구먼, ., ., 솔직히, 재미, 는, 없, 다, ., ., 평...
4         [사이몬페그, 의, 익살, 스런, 연기, 가, 돋보였, 던, 영화, !, 스파이더맨...
                                ...                        
149995           [인간, 이, 문제, 지, ., ., 소, 는, 뭔, 죄, 인가, ., .]
149996                            [평점, 이, 너무, 낮, 아서, ., ..]
149997    [이게, 뭐요, ?, 한국인, 은, 거들먹거리, 고, 필리핀, 혼혈, 은, 착하, ...
149998    [청춘, 영화, 의, 최고봉, ., 방황, 과, 우울, 했, 던, 날, 들, 의, ...
149999             [한국, 영화, 최초, 로, 수간, 하, 는, 내용, 이, 담긴, 영화]
Name: document, Length: 149995, dtype: object

# 불용어 삭제

In [29]:
stopwords = pd.read_csv('./stopword.txt', sep='\n', header=None)

In [30]:
stopword = list(set(list(stopwords[0])))

In [31]:
stopword.extend(['은','는','이','가'])

In [32]:
stop_removed = morphs.progress_apply(lambda x : [ i for i in x if i not in stopword ])

100%|██████████| 149995/149995 [00:13<00:00, 10968.61it/s]


In [33]:
stop_removed

0                             [더, 빙, ., ., 진짜, 짜증, 네요, 목소리]
1         [흠, ., .., 포스터, 보고, 초딩, 영화, 줄, ., ..., 오버, 연기,...
2                                   [너무, 재, 밓었다그래서보는것을추천한다]
3         [교도소, 이야기, 구먼, ., ., 솔직히, 재미, 없, 다, ., ., 평점, 조정]
4         [사이몬페그, 익살, 스런, 연기, 돋보였, 던, 영화, !, 스파이더맨, 늙, 보...
                                ...                        
149995                 [인간, 문제, 지, ., ., 소, 뭔, 죄, 인가, ., .]
149996                               [평점, 너무, 낮, 아서, ., ..]
149997        [이게, 뭐요, ?, 한국인, 거들먹거리, 고, 필리핀, 혼혈, 착하, 다, ?]
149998               [청춘, 영화, 최고봉, ., 방황, 우울, 했, 던, 날, 자화상]
149999                         [한국, 영화, 최초, 수간, 내용, 담긴, 영화]
Name: document, Length: 149995, dtype: object

# 정규표현식으로 한글 제외하고 모두 삭제

In [34]:
import re

In [35]:
kor_text = stop_removed.progress_apply(lambda x : [re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ]', '', i) for i in x])

100%|██████████| 149995/149995 [00:01<00:00, 93735.94it/s] 


In [36]:
kor_text

0                               [더, 빙, , , 진짜, 짜증, 네요, 목소리]
1         [흠, , , 포스터, 보고, 초딩, 영화, 줄, , , 오버, 연기, 가볍, 지,...
2                                   [너무, 재, 밓었다그래서보는것을추천한다]
3             [교도소, 이야기, 구먼, , , 솔직히, 재미, 없, 다, , , 평점, 조정]
4         [사이몬페그, 익살, 스런, 연기, 돋보였, 던, 영화, , 스파이더맨, 늙, 보이...
                                ...                        
149995                     [인간, 문제, 지, , , 소, 뭔, 죄, 인가, , ]
149996                                  [평점, 너무, 낮, 아서, , ]
149997          [이게, 뭐요, , 한국인, 거들먹거리, 고, 필리핀, 혼혈, 착하, 다, ]
149998                [청춘, 영화, 최고봉, , 방황, 우울, 했, 던, 날, 자화상]
149999                         [한국, 영화, 최초, 수간, 내용, 담긴, 영화]
Name: document, Length: 149995, dtype: object

# 빈 string ('') 이 많기 때문에 삭제

In [37]:
clean = kor_text.progress_apply(lambda x : [ i for i in x if i])

100%|██████████| 149995/149995 [00:00<00:00, 346914.90it/s]


In [38]:
clean

0                                   [더, 빙, 진짜, 짜증, 네요, 목소리]
1             [흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 가볍, 지, 않, 구나]
2                                   [너무, 재, 밓었다그래서보는것을추천한다]
3                     [교도소, 이야기, 구먼, 솔직히, 재미, 없, 다, 평점, 조정]
4         [사이몬페그, 익살, 스런, 연기, 돋보였, 던, 영화, 스파이더맨, 늙, 보이, ...
                                ...                        
149995                             [인간, 문제, 지, 소, 뭔, 죄, 인가]
149996                                      [평점, 너무, 낮, 아서]
149997              [이게, 뭐요, 한국인, 거들먹거리, 고, 필리핀, 혼혈, 착하, 다]
149998                  [청춘, 영화, 최고봉, 방황, 우울, 했, 던, 날, 자화상]
149999                         [한국, 영화, 최초, 수간, 내용, 담긴, 영화]
Name: document, Length: 149995, dtype: object

# 데이터 split

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
clean

0                                   [더, 빙, 진짜, 짜증, 네요, 목소리]
1             [흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 가볍, 지, 않, 구나]
2                                   [너무, 재, 밓었다그래서보는것을추천한다]
3                     [교도소, 이야기, 구먼, 솔직히, 재미, 없, 다, 평점, 조정]
4         [사이몬페그, 익살, 스런, 연기, 돋보였, 던, 영화, 스파이더맨, 늙, 보이, ...
                                ...                        
149995                             [인간, 문제, 지, 소, 뭔, 죄, 인가]
149996                                      [평점, 너무, 낮, 아서]
149997              [이게, 뭐요, 한국인, 거들먹거리, 고, 필리핀, 혼혈, 착하, 다]
149998                  [청춘, 영화, 최고봉, 방황, 우울, 했, 던, 날, 자화상]
149999                         [한국, 영화, 최초, 수간, 내용, 담긴, 영화]
Name: document, Length: 149995, dtype: object

In [41]:
train_data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [42]:
x_train, x_test, y_train, y_test = train_test_split(clean, train_data['label'])

# 텍스트 벡터화

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

In [44]:
#한글이기 때문에 lowercase는 false로 지정
cvec = CountVectorizer(tokenizer = lambda x : x, lowercase=False)

In [45]:
#train은 항상 fit_transform으로 
train_vector = cvec.fit_transform(x_train)

In [46]:
#test는 train 기준으로 transform
test_vector = cvec.transform(x_test)

In [47]:
train_vector

<112496x42632 sparse matrix of type '<class 'numpy.int64'>'
	with 1260623 stored elements in Compressed Sparse Row format>

In [48]:
test_vector

<37499x42632 sparse matrix of type '<class 'numpy.int64'>'
	with 411596 stored elements in Compressed Sparse Row format>

# 간단한 classification

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

## Random Forest 활용

In [50]:
rf = RandomForestClassifier(n_estimators = 30, n_jobs=10, verbose=2) # 30개 tree, 10개 core 사용, 학습 progress 보이기

In [51]:
rf.fit(train_vector, y_train)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


building tree 1 of 30building tree 2 of 30

building tree 3 of 30
building tree 4 of 30
building tree 5 of 30
building tree 6 of 30building tree 7 of 30building tree 8 of 30

building tree 9 of 30
building tree 10 of 30

building tree 11 of 30
building tree 12 of 30
building tree 13 of 30
building tree 14 of 30
building tree 15 of 30
building tree 16 of 30
building tree 17 of 30
building tree 18 of 30
building tree 19 of 30
building tree 20 of 30
building tree 21 of 30
building tree 22 of 30
building tree 23 of 30
building tree 24 of 30
building tree 25 of 30
building tree 26 of 30
building tree 27 of 30
building tree 28 of 30
building tree 29 of 30
building tree 30 of 30


[Parallel(n_jobs=10)]: Done  27 out of  30 | elapsed:   10.7s remaining:    1.2s
[Parallel(n_jobs=10)]: Done  30 out of  30 | elapsed:   11.3s finished


RandomForestClassifier(n_estimators=30, n_jobs=10, verbose=2)

In [52]:
y_pred_rf = rf.predict(test_vector) # 예측하기

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  27 out of  30 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=10)]: Done  30 out of  30 | elapsed:    0.1s finished


## metric으로 계산
### balanced == accuracy_score
### imbalanced == f1_Score

In [72]:
from sklearn.metrics import accuracy_score, f1_score

In [73]:
accuracy_score(y_test, y_pred_rf), f1_score(y_test ,y_pred_rf)

(0.8212752340062401, 0.8174737186121249)